In [1]:
"""
# 🏈 NFL Big Data Bowl 2026 - Geometric Rules Baseline

## 📊 Performance
- **Public Leaderboard**: 2.897 yards RMSE
- **Execution Time**: <5 seconds  
- **Approach**: Physics-based geometric rules

## 🎯 Key Insights

### 1. Targeted Receivers
Players with the ball thrown to them run directly toward the **ball landing point**.

### 2. Defensive Coverage  
Defenders mirror the receiver they're guarding, maintaining a **distance-based offset**.

### 3. Linear Interpolation
Movement between frames is approximated linearly based on time progress.

## 💡 Why This Works

Simple geometric rules capture the essential movement patterns:
- Receivers know where the ball will land → direct path
- Defenders track receivers → maintain spatial relationship
- Short time horizons → linear approximation is reasonable

## 🚀 Results

This simple baseline achieves competitive results without any machine learning, 
demonstrating the power of domain knowledge.

Perfect starting point for newcomers to the competition!

---

## 📖 Code Structure

1. **Velocity Calculation**: Convert speed/direction to x/y components
2. **Endpoint Prediction**: Rules-based final position
3. **Linear Interpolation**: Smooth movement across frames
4. **Field Constraints**: Clip to valid coordinates

Let's dive into the implementation! 👇
"""

"\n# 🏈 NFL Big Data Bowl 2026 - Geometric Rules Baseline\n\n## 📊 Performance\n- **Public Leaderboard**: 2.897 yards RMSE\n- **Execution Time**: <5 seconds  \n- **Approach**: Physics-based geometric rules\n\n## 🎯 Key Insights\n\n### 1. Targeted Receivers\nPlayers with the ball thrown to them run directly toward the **ball landing point**.\n\n### 2. Defensive Coverage  \nDefenders mirror the receiver they're guarding, maintaining a **distance-based offset**.\n\n### 3. Linear Interpolation\nMovement between frames is approximated linearly based on time progress.\n\n## 💡 Why This Works\n\nSimple geometric rules capture the essential movement patterns:\n- Receivers know where the ball will land → direct path\n- Defenders track receivers → maintain spatial relationship\n- Short time horizons → linear approximation is reasonable\n\n## 🚀 Results\n\nThis simple baseline achieves competitive results without any machine learning, \ndemonstrating the power of domain knowledge.\n\nPerfect starting 

In [2]:
"""
NFL Big Data Bowl 2026 - Geometric Rules Baseline
Simple yet effective approach using physics and game theory

Public LB: 2.897 yards RMSE
Execution Time: <5 seconds

Key Insights:
1. Targeted Receivers → Ball landing point
2. Defensive Coverage → Mirror receivers (maintain offset)
3. Linear interpolation across frames
4. Distance-based offset adjustment

This notebook demonstrates that simple geometric rules can achieve 
competitive results without complex models.
"""

import os
import pandas as pd
import polars as pl
import numpy as np
import kaggle_evaluation.nfl_inference_server

# ============================================================================
# APPROACH OVERVIEW
# ============================================================================
"""
Our approach is based on two simple observations:

1. **Targeted Receivers** always run toward the ball landing point
   - They know where the ball will land
   - Direct linear path provides good approximation

2. **Defensive Coverage** mirrors the receiver they're guarding
   - Maintain initial spatial offset
   - Adjust offset based on distance (closer = tighter coverage)

This gives us a geometric baseline that captures the essential 
player movement patterns without machine learning.
"""

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
    """
    Predict player positions using geometric rules
    
    Parameters:
    - test: Target predictions (game_id, play_id, nfl_id, frame_id)
    - test_input: Historical tracking data (pre-throw frames)
    
    Returns:
    - DataFrame with predicted (x, y) positions
    """
    test_pd = test.to_pandas()
    test_input_pd = test_input.to_pandas()
    
    # Extract players we need to predict
    predict_players = test_input_pd[test_input_pd['player_to_predict'] == True]
    
    # Get throw moment (last frame before ball is thrown)
    throw_moment = predict_players.groupby(['game_id', 'play_id', 'nfl_id']).last().reset_index()
    
    # ========================================================================
    # Step 1: Calculate velocity vectors
    # ========================================================================
    throw_moment['vx'] = throw_moment['s'] * np.cos(np.radians(throw_moment['dir']))
    throw_moment['vy'] = throw_moment['s'] * np.sin(np.radians(throw_moment['dir']))
    
    # ========================================================================
    # Step 2: Set starting position
    # ========================================================================
    throw_moment['start_x'] = throw_moment['x']
    throw_moment['start_y'] = throw_moment['y']
    
    # ========================================================================
    # Step 3: Calculate time horizon
    # ========================================================================
    time_horizon = throw_moment['num_frames_output'] / 10.0  # frames to seconds
    
    # ========================================================================
    # Step 4: Default endpoint = momentum-based prediction
    # ========================================================================
    throw_moment['end_x'] = throw_moment['start_x'] + throw_moment['vx'] * time_horizon
    throw_moment['end_y'] = throw_moment['start_y'] + throw_moment['vy'] * time_horizon
    
    # ========================================================================
    # Step 5: Override for Targeted Receivers → Ball landing point
    # ========================================================================
    receiver_mask = throw_moment['player_role'] == 'Targeted Receiver'
    throw_moment.loc[receiver_mask, 'end_x'] = throw_moment.loc[receiver_mask, 'ball_land_x']
    throw_moment.loc[receiver_mask, 'end_y'] = throw_moment.loc[receiver_mask, 'ball_land_y']
    
    # ========================================================================
    # Step 6: Defensive Coverage mirrors receivers
    # ========================================================================
    coverage_mask = throw_moment['player_role'] == 'Defensive Coverage'
    
    for idx, row in throw_moment[coverage_mask].iterrows():
        # Find receiver in same play
        same_play = throw_moment[
            (throw_moment['game_id'] == row['game_id']) & 
            (throw_moment['play_id'] == row['play_id']) & 
            (throw_moment['player_role'] == 'Targeted Receiver')
        ]
        
        if len(same_play) > 0:
            receiver = same_play.iloc[0]
            
            # Calculate initial offset from receiver
            offset_x = row['start_x'] - receiver['start_x']
            offset_y = row['start_y'] - receiver['start_y']
            initial_distance = np.sqrt(offset_x**2 + offset_y**2)
            
            # Distance-based offset factor
            # Closer defenders stick tighter to receivers
            if initial_distance < 5:
                offset_factor = 0.8  # Tight coverage
            elif initial_distance < 10:
                offset_factor = 0.6  # Medium coverage
            else:
                offset_factor = 0.4  # Zone coverage
            
            # Final position = receiver's endpoint + scaled offset
            throw_moment.at[idx, 'end_x'] = receiver['end_x'] + offset_x * offset_factor
            throw_moment.at[idx, 'end_y'] = receiver['end_y'] + offset_y * offset_factor
    
    # ========================================================================
    # Step 7: Clip to field boundaries
    # ========================================================================
    throw_moment['end_x'] = throw_moment['end_x'].clip(0, 120)
    throw_moment['end_y'] = throw_moment['end_y'].clip(0, 53.3)
    
    # ========================================================================
    # Step 8: Merge with test data
    # ========================================================================
    result = test_pd.merge(
        throw_moment[['game_id', 'play_id', 'nfl_id', 'start_x', 'start_y', 'end_x', 'end_y', 'num_frames_output']],
        on=['game_id', 'play_id', 'nfl_id'],
        how='left'
    )
    
    # ========================================================================
    # Step 9: Linear interpolation across frames
    # ========================================================================
    # Progress: 0 at frame_id=1, 1 at frame_id=num_frames_output
    progress = (result['frame_id'] - 1) / result['num_frames_output'].clip(lower=1)
    progress = progress.clip(0, 1)
    
    result['predicted_x'] = result['start_x'] + progress * (result['end_x'] - result['start_x'])
    result['predicted_y'] = result['start_y'] + progress * (result['end_y'] - result['start_y'])
    
    # ========================================================================
    # Step 10: Final safety checks
    # ========================================================================
    result['predicted_x'] = result['predicted_x'].clip(0, 120)
    result['predicted_y'] = result['predicted_y'].clip(0, 53.3)
    
    # Handle any missing predictions (shouldn't happen)
    result['predicted_x'] = result['predicted_x'].fillna(60.0)
    result['predicted_y'] = result['predicted_y'].fillna(26.65)
    
    # ========================================================================
    # Return predictions
    # ========================================================================
    predictions = pl.DataFrame({
        'x': result['predicted_x'].values,
        'y': result['predicted_y'].values
    })
    
    assert len(predictions) == len(test), f"Prediction count mismatch: {len(predictions)} vs {len(test)}"
    
    return predictions

# ============================================================================
# Inference Server Setup
# ============================================================================

inference_server = kaggle_evaluation.nfl_inference_server.NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))

"""
============================================================================
RESULTS & INSIGHTS
============================================================================

Public Leaderboard: 2.897 yards RMSE

Key Takeaways:
1. Simple geometric rules capture essential movement patterns
2. Distance-based offset adjustment significantly improves defender predictions
3. Linear interpolation is surprisingly effective for short time horizons
4. No complex models needed for competitive baseline

Future Improvements:
- Add acceleration-based trajectory adjustments
- Consider field zones (red zone behavior differs)
- Account for multiple receivers (coverage switching)
- Use player speed to adjust interpolation (faster = less linear)

This baseline provides a strong foundation for more complex approaches.
============================================================================
"""

'\n============================================================================\nRESULTS & INSIGHTS\n============================================================================\n\nPublic Leaderboard: 2.897 yards RMSE\n\nKey Takeaways:\n1. Simple geometric rules capture essential movement patterns\n2. Distance-based offset adjustment significantly improves defender predictions\n3. Linear interpolation is surprisingly effective for short time horizons\n4. No complex models needed for competitive baseline\n\nFuture Improvements:\n- Add acceleration-based trajectory adjustments\n- Consider field zones (red zone behavior differs)\n- Account for multiple receivers (coverage switching)\n- Use player speed to adjust interpolation (faster = less linear)\n\nThis baseline provides a strong foundation for more complex approaches.\n============================================================================\n'